<a href="https://colab.research.google.com/github/SmritiReddyy/MLOps_OCR_Only/blob/main/Latest_ENG_SPA_Translation7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

# ============================================================
# CELL 1: INSTALL ALL DEPENDENCIES
# ============================================================
%%time

!pip uninstall -y numpy paddlepaddle paddleocr torch transformers sentencepiece 2>/dev/null
!pip install numpy==1.26.4
!pip install -q pymupdf Pillow
!pip install -q pytesseract
!apt-get install -qq tesseract-ocr tesseract-ocr-spa tesseract-ocr-por > /dev/null 2>&1
!pip install paddlepaddle==2.6.2
!pip install paddleocr==2.8.1
!pip install torch==2.2.2
!pip install transformers==4.38.2 sentencepiece
!pip install langchain==0.0.352
!pip install -q langchain-community
!pip install -q groq
!pip install -q presidio-analyzer presidio-anonymizer
!python -m spacy download en_core_web_lg -q

print("\n" + "="*60)
print("✅ All tools installed successfully!")
print("="*60)


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: paddlepaddle 2.6.2
Uninstalling paddlepaddle-2.6.2:
  Successfully uninstalled paddlepaddle-2.6.2
Found existing installation: paddleocr 2.8.1
Uninstalling paddleocr-2.8.1:
  Successfully uninstalled paddleocr-2.8.1
Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
Found existing installation: sentencepiece 0.2.1
Uninstalling sentencepiece-0.2.1:
  Successfully uninstalled sentencepiece-0.2.1
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.25.0+cu128 requires to

  Using cached paddlepaddle-2.6.2-cp312-cp312-manylinux1_x86_64.whl.metadata (8.6 kB)
Using cached paddlepaddle-2.6.2-cp312-cp312-manylinux1_x86_64.whl (125.9 MB)


  Using cached paddleocr-2.8.1-py3-none-any.whl.metadata (19 kB)
Using cached paddleocr-2.8.1-py3-none-any.whl (407 kB)


  Using cached torch-2.2.2-cp312-cp312-manylinux1_x86_64.whl.metadata (25 kB)
Using cached torch-2.2.2-cp312-cp312-manylinux1_x86_64.whl (755.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.1 requires transformers, which is not installed.
sentence-transformers 5.2.3 requires transformers<6.0.0,>=4.41.0, which is not installed.
torchtune 0.6.1 requires sentencepiece, which is not installed.
torchaudio 2.10.0+cu128 requires torch==2.10.0, but you have torch 2.2.2 which is incompatible.
torchvision 0.25.0+cu128 requires torch==2.10.0, but you have torch 2.2.2 which is incompatible.


  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached sentencepiece-0.2.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (10 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
Using cached sentencepiece-0.2.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (1.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.2.3 requires transformers<6.0.0,>=4.41.0, but you have transformers 4.38.2 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.

✅ All tools installed successfully!
CPU times: user 18.6 s, sys: 2.53 s, total: 21.1 s
Wall time: 1min 50s


In [1]:

# ============================================================
# CELL 2: UPLOAD PDF
# ============================================================
import os
from pathlib import Path

WORK_DIR = Path("/content/courtaccess_benchmark")
WORK_DIR.mkdir(exist_ok=True)

from google.colab import files
print("Upload a PDF to translate:")

try:
    uploaded = files.upload()
    if uploaded:
        for fname, content in uploaded.items():
            pdf_path = WORK_DIR / fname
            pdf_path.write_bytes(content)
            print(f"✅ Uploaded: {pdf_path}")
            INPUT_PDF = str(pdf_path)
except KeyboardInterrupt:
    print("Skipped upload.")
    INPUT_PDF = None

# Safety check — never translate a previous output
assert INPUT_PDF and "unified_reconstructed" not in INPUT_PDF, \
    "❌ INPUT_PDF is pointing to a previous output! Re-upload your original PDF."



Upload a PDF to translate:


Saving test4.pdf to test4.pdf
✅ Uploaded: /content/courtaccess_benchmark/test4.pdf


In [2]:

# ============================================================
# CELL 3: NLLB TRANSLATION SETUP
# ============================================================
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda")
MODEL_NAME = "facebook/nllb-200-distilled-1.3B"

print("⏳ Loading NLLB model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
print("✅ NLLB ready")

SOURCE_LANG = "eng_Latn"
TARGET_LANG = "spa_Latn"

# ── Patterns that should NEVER be translated ──────────────────
SKIP_PATTERNS = [
    r'^[\d\s\.\-\/\(\)\\,]+$',           # pure numbers/dates/punctuation
    r'^\s*[xX]\s*$',                       # signature placeholder "X"
    r'^https?://',                          # URLs
    r'^\$[\d,\.]+$',                        # dollar amounts
    r'^[\d]{1,2}[\/\-][\d]{1,2}[\/\-][\d]{2,4}$',  # date formats
    r'^[\-\s]*$',                           # dashes/spaces only
    r'^\[[\d\.\(\)a-zA-Z\;\s]+\]',        # [14.1(a)] style citations
    r'^\d+\.\s*$',                          # standalone "1." "2."
    r'^[_#\s]+$',                           # blank fill lines
    r'^Mass\.\s+R\.',                       # Mass. R. Crim. P. citations
    r'^\d+:\d+-[a-zA-Z]+-\d+',            # case numbers like 3:11-cr-1234
]

def should_translate(text: str) -> bool:
    t = text.strip()
    if not t:
        return False
    if len(t) <= 3:
        return False
    if t.isupper() and len(t) < 12:
        return False
    for pat in SKIP_PATTERNS:
        if re.match(pat, t):
            return False
    return True


def split_into_sentences(text: str) -> list:
    parts = re.split(r'(?<=[.!?])\s+', text.strip())
    return [p for p in parts if p.strip()]


def _raw_batch_translate(texts: list) -> list:
    tokenizer.src_lang = SOURCE_LANG
    encoded = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    input_len = encoded["input_ids"].shape[1]
    max_out = min(512, input_len + 40)

    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(TARGET_LANG),
        max_length=max_out,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


def batch_translate(texts: list) -> list:
    results = []
    for text in texts:
        if not should_translate(text):
            results.append(text)
            continue

        sentences = split_into_sentences(text)
        if len(sentences) <= 1:
            translated_list = _raw_batch_translate([text])
            translated = translated_list[0]
        else:
            translated_parts = _raw_batch_translate(sentences)
            translated = " ".join(translated_parts)

        ratio = len(translated) / max(len(text), 1)
        if ratio > 2.8 or ratio < 0.15:
            print(f"   ⚠️  Hallucination suspected (ratio={ratio:.1f}), keeping original: '{text[:60]}'")
            results.append(text)
        else:
            results.append(translated)

    return results



⏳ Loading NLLB model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possibl

✅ NLLB ready


In [3]:
#============================================================
# CELL 4: GROQ / LLaMA 3.3 70B LEGAL VERIFICATION
# ============================================================
import time
import json
from groq import Groq
from google.colab import userdata

GROQ_API_KEY = userdata.get('GROQ_API_KEY')
groq_client = Groq(api_key=GROQ_API_KEY)

_groq_request_times = []

def _wait_for_rate_limit(max_per_minute=25):
    now = time.time()
    global _groq_request_times
    _groq_request_times = [t for t in _groq_request_times if now - t < 60]
    if len(_groq_request_times) >= max_per_minute:
        wait = 60 - (now - _groq_request_times[0]) + 1
        print(f"   ⏳ Groq rate limit — waiting {wait:.1f}s...")
        time.sleep(wait)
    _groq_request_times.append(time.time())


def _strip_code_fences(text: str) -> str:
    fence = chr(96) * 3
    text = text.strip()
    if text.startswith(fence + "json"):
        text = text[7:]
    elif text.startswith(fence):
        text = text[3:]
    if text.endswith(fence):
        text = text[:-3]
    return text.strip()


HIGH_RISK_PATTERNS = [
    r'Mass\.\s+R\.',
    r'\[[\d\.\(\)a-zA-Z\;\s]+\]',
    r'\bCommonwealth\b',
    r'\bwaiver\b|\bwaive\b',
    r'\bdefendant\b|\bplaintiff\b',
    r'\bdocket\b',
    r'G\.L\.\s+c\.',
    r'Paragraph\s+\d+',
    r'§\s*\d+',
    r'^\d+\.',
    r'\breciprocal\b',
    r'\bautomatic\b',
    r'\bARRAIGNMENT\b',
    r'\bplea\b|\bplead\b',
]

def is_high_risk(original: str) -> bool:
    for pat in HIGH_RISK_PATTERNS:
        if re.search(pat, original, re.IGNORECASE):
            return True
    return False


def verify_legal_translation(original_en: str, translated_es: str) -> dict:
    """Single-span verification — used only for the connection test."""
    if len(original_en.strip()) < 20:
        return {"approved": True, "issues": [], "corrected": translated_es}
    _wait_for_rate_limit()
    prompt = (
        "You are a legal translation verifier for Massachusetts court documents.\n"
        "Check if this Spanish translation of English legal text is accurate.\n\n"
        f"ENGLISH: {original_en}\n"
        f"SPANISH: {translated_es}\n\n"
        "Respond in this exact JSON format with no extra text:\n"
        '{"approved": true or false, "issues": ["issue 1"], '
        '"corrected": "corrected Spanish or same as input if approved"}'
    )
    try:
        response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_tokens=512,
        )
        raw = _strip_code_fences(response.choices[0].message.content)
        return json.loads(raw)
    except Exception as e:
        print(f"   ⚠️ Groq error: {e}")
        return {"approved": True, "issues": [], "corrected": translated_es}


def verify_page_translations(original_spans: list, translated_spans: list,
                              batch_size: int = 8) -> list:
    verified = list(translated_spans)
    high_risk_indices = [
        i for i, orig in enumerate(original_spans)
        if should_translate(orig) and is_high_risk(orig)
    ]
    if not high_risk_indices:
        print(f"   ✅ No high-risk spans found, skipping Groq")
        return verified

    print(f"   🔍 {len(high_risk_indices)}/{len(original_spans)} spans flagged as high-risk")

    for batch_start in range(0, len(high_risk_indices), batch_size):
        batch_indices = high_risk_indices[batch_start:batch_start + batch_size]
        orig_batch = [original_spans[i] for i in batch_indices]
        trans_batch = [translated_spans[i] for i in batch_indices]
        _wait_for_rate_limit()

        pairs = "\n".join(
            f"{k+1}. EN: {o}\n   ES: {t}"
            for k, (o, t) in enumerate(zip(orig_batch, trans_batch))
        )
        prompt = (
            "You are a legal translator verifying English to Spanish translations "
            "of Massachusetts court documents.\n"
            "For each pair, check if the Spanish is legally accurate.\n"
            "Pay special attention to: legal citations (Mass. R. Crim. P.), "
            "legal terms (Commonwealth, defendant, waiver, docket), "
            "section references (G.L. c. X, § Y), and paragraph numbers.\n\n"
            "Return ONLY a JSON array of corrected Spanish strings, one per pair, "
            "in the same order. If a translation is already correct, return it unchanged.\n"
            "No explanation, no markdown, just the JSON array.\n\n"
            f"PAIRS:\n{pairs}\n\n"
            'Return format: ["corrected span 1", "corrected span 2", ...]'
        )
        try:
            response = groq_client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1,
                max_tokens=1024,
            )
            raw = _strip_code_fences(response.choices[0].message.content)
            corrected_list = json.loads(raw)
            if isinstance(corrected_list, list) and len(corrected_list) == len(batch_indices):
                for idx, corrected in zip(batch_indices, corrected_list):
                    if corrected and corrected.strip():
                        ratio = len(corrected) / max(len(translated_spans[idx]), 1)
                        if 0.2 <= ratio <= 4.0:
                            verified[idx] = corrected.strip()
                print(f"   ✅ Batch {batch_start//batch_size + 1}: corrected {len(batch_indices)} spans")
            else:
                print(f"   ⚠️  Batch {batch_start//batch_size + 1}: response length mismatch, keeping NLLB")
        except Exception as e:
            print(f"   ⚠️  Groq error on batch: {e} — keeping NLLB output")

    return verified


print("🧪 Testing Groq connection...")
test = verify_legal_translation(
    "The defendant waives the right to jury trial.",
    "El acusado renuncia al derecho a juicio con jurado."
)
print(f"   approved={test['approved']}, issues={test['issues']}")
print("✅ Groq ready" if test['approved'] else f"⚠️ Groq flagged: {test['issues']}")



🧪 Testing Groq connection...
   approved=True, issues=[]
✅ Groq ready


In [4]:

# ============================================================
# CELL 5: PRESIDIO PII REDACTION
# ============================================================
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig

print("⏳ Loading Presidio...")
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()
print("✅ Presidio ready")

# ── Toggle PII redaction on/off ───────────────────────────
# Set True for batch processing or storage scenarios
# Set False for personal use (person translating their own form)
REDACT_PII = True
REDACT_DATES = False  # keep dates visible in output

PII_ENTITIES = [
    "PERSON",
    "PHONE_NUMBER",
    "EMAIL_ADDRESS",
    "US_SSN",
    "CREDIT_CARD",
    "US_PASSPORT",
    "US_DRIVER_LICENSE",
    "IP_ADDRESS",
    "DATE_TIME",
]
if not REDACT_DATES and "DATE_TIME" in PII_ENTITIES:
    PII_ENTITIES.remove("DATE_TIME")

PRESERVE_PATTERNS = [
    r'G\.L\.\s+c\.\s+\d+',
    r'§\s*\d+[A-Z]?',
    r'\[\d+[\.\(\)a-zA-Z\;]+\]',
    r'Mass\.\s+R\.\s+Crim\.',
    r'Paragraph\s+\d+',
    r'TC\d+',
    r'P\.\s*\d+\.\d+',
]

def should_preserve(text: str) -> bool:
    for pat in PRESERVE_PATTERNS:
        if re.search(pat, text, re.IGNORECASE):
            return True
    return False


def redact_pii(text: str, language: str = "en") -> tuple:
    if not REDACT_PII or not text.strip() or should_preserve(text):
        return text, []
    try:
        results = analyzer.analyze(text=text, entities=PII_ENTITIES, language="en")
    except Exception:
        return text, []
    if not results:
        return text, []
    found_entities = list(set(r.entity_type for r in results))
    anonymized = anonymizer.anonymize(
        text=text,
        analyzer_results=results,
        operators={
            entity: OperatorConfig("replace", {"new_value": f"[{entity}]"})
            for entity in found_entities
        }
    )
    return anonymized.text, found_entities


def redact_spans(original_texts: list, translated_texts: list) -> list:
    if not REDACT_PII:
        return translated_texts
    redacted = list(translated_texts)
    total_redacted = 0
    for i, (orig, trans) in enumerate(zip(original_texts, translated_texts)):
        _, entities = redact_pii(orig, language="en")
        if not entities:
            continue
        redacted_trans, _ = redact_pii(trans, language="en")
        redacted[i] = redacted_trans
        total_redacted += 1
        print(f"   🔒 Span {i}: redacted {entities} → '{trans[:40]}...' → '{redacted_trans[:40]}...'")
    if total_redacted == 0:
        print(f"   ✅ No PII found in {len(original_texts)} spans")
    else:
        print(f"   🔒 Redacted PII in {total_redacted}/{len(original_texts)} spans")
    return redacted


def redact_form_fields(page) -> int:
    """Redact PII in PDF form field values (widget annotations)."""
    if not REDACT_PII:
        return 0
    widgets = list(page.widgets())
    if not widgets:
        return 0
    redacted_count = 0
    for widget in widgets:
        field_value = widget.field_value
        if not field_value or not str(field_value).strip():
            continue
        value_str = str(field_value).strip()
        redacted_value, entities = redact_pii(value_str, language="en")
        if entities:
            print(f"   🔒 Form field '{widget.field_name}': '{value_str[:30]}' → '{redacted_value[:30]}' {entities}")
            widget.field_value = redacted_value
            widget.update()
            rect = widget.rect
            if rect:
                page.add_redact_annot(rect, fill=(1, 1, 1))
            redacted_count += 1
    if redacted_count > 0:
        page.apply_redactions(images=pymupdf.PDF_REDACT_IMAGE_NONE)
    return redacted_count


print("\n🧪 Testing Presidio...")
test_cases = [
    "Sunny Yadav, Aditi Putrevu, Devasai Choppa",
    "Phone No: 8573519565",
    "75 Saint Alphonsus St, Apt 1215, Boston, MA 02120",
    "G.L. c. 263, § 6",
    "Mass. R. Crim. P. 14.2",
]
for t in test_cases:
    redacted, entities = redact_pii(t)
    status = "🔒" if entities else "✅"
    print(f"   {status} '{t[:50]}' → '{redacted[:50]}' {entities if entities else ''}")



⏳ Loading Presidio...


✅ Presidio ready

🧪 Testing Presidio...
   🔒 'Sunny Yadav, Aditi Putrevu, Devasai Choppa' → '[PERSON], [PERSON], [PERSON]' ['PERSON']
   🔒 'Phone No: 8573519565' → 'Phone No: [PHONE_NUMBER]' ['US_DRIVER_LICENSE', 'PHONE_NUMBER']
   🔒 '75 Saint Alphonsus St, Apt 1215, Boston, MA 02120' → '75 [PERSON], Apt 1215, Boston, MA 02120' ['PERSON']
   ✅ 'G.L. c. 263, § 6' → 'G.L. c. 263, § 6' 
   ✅ 'Mass. R. Crim. P. 14.2' → 'Mass. R. Crim. P. 14.2' 


In [5]:

# ============================================================
# CELL 6: CORE PIPELINE — OCR, RECONSTRUCTION, MAIN LOOP
# ============================================================
import os
import sys
import pymupdf
from paddleocr import PaddleOCR

os.environ["PADDLE_PDX_DISABLE_MODEL_SOURCE_CHECK"] = "True"

try:
    if 'paddlex.repo_manager.core' in sys.modules:
        import paddlex.repo_manager.core as pdx_core
        if hasattr(pdx_core, '_GlobalContext'):
            pdx_core._GlobalContext._initialized = False
except Exception:
    pass

PaddleOCR = None
try:
    from paddleocr import PaddleOCR
    print(f"✅ PaddleOCR: available")
except Exception as e:
    PaddleOCR = None
    print(f"⚠️ PaddleOCR unavailable: {e}")

OCR_DPI = 300
CONFIDENCE_THRESHOLD = 0.80

font_map = {
    "Helvetica": "helv", "Helvetica-Bold": "hebo",
    "Helvetica-Oblique": "heit", "Helvetica-BoldOblique": "hebi",
    "Times-Roman": "tiro", "Times-Bold": "tibo",
    "Times-Italic": "tiit", "Times-BoldItalic": "tibi",
    "Courier": "cour", "Courier-Bold": "cobo",
    "Courier-Oblique": "coit", "Courier-BoldOblique": "cobi",
    "Symbol": "symb", "ZapfDingbats": "zadb",
}

# ── Helper Functions ──────────────────────────────────────

def get_font_code(span, font_map):
    fontname = span.get("font", "")
    flags = span.get("flags", 0)
    direct = font_map.get(fontname)
    if direct:
        return direct
    is_bold = bool(flags & 16) or "bold" in fontname.lower()
    is_italic = bool(flags & 2) or "italic" in fontname.lower() or "oblique" in fontname.lower()
    is_mono = bool(flags & 8) or "courier" in fontname.lower() or "mono" in fontname.lower()
    is_serif = bool(flags & 4) or "times" in fontname.lower() or "serif" in fontname.lower()
    if is_mono:
        if is_bold and is_italic: return "cobi"
        if is_bold: return "cobo"
        if is_italic: return "coit"
        return "cour"
    elif is_serif:
        if is_bold and is_italic: return "tibi"
        if is_bold: return "tibo"
        if is_italic: return "tiit"
        return "tiro"
    else:
        if is_bold and is_italic: return "hebi"
        if is_bold: return "hebo"
        if is_italic: return "heit"
        return "helv"


def get_background_color(page, bbox):
    """Sample the background color behind a text span."""
    try:
        clip = pymupdf.Rect(bbox)
        pix = page.get_pixmap(clip=clip, matrix=pymupdf.Matrix(1, 1))
        samples = pix.samples
        if len(samples) >= 3:
            r, g, b = samples[0]/255, samples[1]/255, samples[2]/255
            return (r, g, b)
    except Exception:
        pass
    return (1, 1, 1)


def fit_fontsize(text, original_size, bbox_width, fontname="helv", min_size=4.0):
    """Scale font down until text fits within bbox_width with 5% buffer."""
    font = pymupdf.Font(fontname)
    size = original_size
    while size >= min_size:
        w = font.text_length(text, fontsize=size)
        if w <= bbox_width * 0.95:
            return size
        size -= 0.5
    return min_size


def analyze_page(page):
    """Classify page as DIGITAL, SCANNED, or OCR'd SCAN."""
    td = page.get_text("dict", flags=pymupdf.TEXTFLAGS_TEXT)
    span_count = 0
    img_blocks = 0
    for b in td["blocks"]:
        if b["type"] == 1:
            img_blocks += 1
        elif b["type"] == 0:
            for line in b["lines"]:
                for span in line["spans"]:
                    if span["text"].strip():
                        span_count += 1

    images = page.get_images(full=True)
    drawings = page.get_drawings()
    page_area = page.rect.width * page.rect.height
    total_img_area = 0

    for img in images:
        xref = img[0]
        try:
            rects = page.get_image_rects(xref)
            for r in rects:
                total_img_area += r.width * r.height
        except:
            pass

    if total_img_area == 0:
        for b in td["blocks"]:
            if b["type"] == 1:
                bbox = b["bbox"]
                total_img_area += (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])

    has_large_image = False
    if total_img_area == 0 and len(images) > 0:
        for img in images:
            img_w, img_h = img[2], img[3]
            if img_w > 1000 and img_h > 1000:
                has_large_image = True
                total_img_area = page_area * 0.9
                break

    img_coverage = total_img_area / page_area if page_area > 0 else 0
    is_scanned = False
    page_type = "DIGITAL"

    if span_count < 10 and len(images) > 0:
        is_scanned = True
        page_type = "SCANNED"
    elif img_coverage > 0.5 and len(drawings) < 5:
        is_scanned = True
        page_type = "OCR'd SCAN" if span_count >= 10 else "SCANNED"
    elif len(drawings) < 5 and span_count > 10 and (has_large_image or len(images) > 0):
        is_scanned = True
        page_type = "OCR'd SCAN"

    return {
        "is_scanned": is_scanned,
        "page_type": page_type,
        "span_count": span_count,
        "images": len(images),
        "img_blocks": img_blocks,
        "drawings": len(drawings),
        "img_coverage": round(img_coverage * 100, 1),
        "has_large_image": has_large_image,
    }


def is_content_image(page, info):
    """Returns True if page is image-only with no meaningful text (photo, diagram)."""
    return info["img_coverage"] > 80 and info["span_count"] < 5


# ── Page Reconstruction ───────────────────────────────────

def reconstruct_digital_page(out_page, font_map, use_verification=True):
    text_dict = out_page.get_text("dict", flags=pymupdf.TEXTFLAGS_TEXT)
    all_spans = []
    for block in text_dict["blocks"]:
        if block["type"] != 0:
            continue
        for line in block["lines"]:
            for span in line["spans"]:
                if span["text"].strip():
                    all_spans.append(span)

    if not all_spans:
        form_redacted = redact_form_fields(out_page)
        if form_redacted:
            print(f"   🔒 Redacted {form_redacted} form field(s)")
        return 0

    # Step 1: Translate
    original_texts = [span["text"] for span in all_spans]
    translated_texts = batch_translate(original_texts)

    # Step 2: Groq legal verification (high-risk spans only)
    if use_verification:
        print(f"   🔍 Running Groq verification on {len(all_spans)} spans...")
        translatable_indices = [
            i for i, t in enumerate(original_texts) if should_translate(t)
        ]
        orig_to_verify = [original_texts[i] for i in translatable_indices]
        trans_to_verify = [translated_texts[i] for i in translatable_indices]
        if orig_to_verify:
            verified = verify_page_translations(orig_to_verify, trans_to_verify, batch_size=8)
            for idx, verified_text in zip(translatable_indices, verified):
                translated_texts[idx] = verified_text

    # Step 3: Presidio PII redaction on text spans
    print(f"   🔒 Running PII redaction on spans...")
    translated_texts = redact_spans(original_texts, translated_texts)

    # Step 4: Presidio PII redaction on form fields
    form_redacted = redact_form_fields(out_page)
    if form_redacted:
        print(f"   🔒 Redacted {form_redacted} form field(s)")

    # Step 5: Redact originals (preserve background color)
    for span in all_spans:
        bg_color = get_background_color(out_page, span["bbox"])
        out_page.add_redact_annot(pymupdf.Rect(span["bbox"]), fill=bg_color)
    out_page.apply_redactions(images=pymupdf.PDF_REDACT_IMAGE_NONE)

    # Step 6: Reinsert translated text
    count = 0
    for span, text in zip(all_spans, translated_texts):
        if not text.strip():
            continue
        bbox = pymupdf.Rect(span["bbox"])
        color = pymupdf.sRGB_to_rgb(span["color"])
        fnt = get_font_code(span, font_map)
        original_size = span["size"]
        fitted_size = fit_fontsize(text, original_size, bbox.width, fnt)
        origin = pymupdf.Point(span["origin"])
        if fitted_size < original_size * 0.6:
            origin = pymupdf.Point(bbox.x0, bbox.y0 + bbox.height * 0.78)
        try:
            out_page.insert_text(origin, text, fontsize=fitted_size, fontname=fnt, color=color)
            count += 1
        except Exception:
            try:
                out_page.insert_text(origin, text, fontsize=fitted_size, fontname="helv", color=color)
                count += 1
            except Exception:
                pass

    return count


def reconstruct_scanned_page(out_page, ocr_engine, page_num_for_log):
    zoom = OCR_DPI / 72
    pix = out_page.get_pixmap(matrix=pymupdf.Matrix(zoom, zoom))
    img_bytes = pix.tobytes("png")

    import tempfile
    tmp = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    tmp.write(img_bytes)
    tmp.close()

    result = ocr_engine.ocr(tmp.name, cls=True)
    os.unlink(tmp.name)

    detections = result[0] if result and result[0] else []
    if not detections:
        return 0

    page_w, page_h = out_page.rect.width, out_page.rect.height
    scale_x = page_w / pix.width
    scale_y = page_h / pix.height

    regions = []
    for det in detections:
        bbox_px = det[0]
        text = det[1][0]
        conf = det[1][1]
        if conf < CONFIDENCE_THRESHOLD or len(text.strip()) < 3:
            continue
        xs = [p[0] for p in bbox_px]
        ys = [p[1] for p in bbox_px]
        rect = pymupdf.Rect(
            min(xs) * scale_x - 1, min(ys) * scale_y - 1,
            max(xs) * scale_x + 1, max(ys) * scale_y + 1,
        )
        regions.append({"rect": rect, "text": text})

    if not regions:
        return 0

    # White-out detected regions
    shape = out_page.new_shape()
    for r in regions:
        shape.draw_rect(r["rect"])
        shape.finish(color=None, fill=(1, 1, 1))
    shape.commit(overlay=True)

    # Translate
    original_texts = [r["text"] for r in regions]
    translated_texts = batch_translate(original_texts)

    # PII redaction
    translated_texts = redact_spans(original_texts, translated_texts)

    # Insert translated text
    count = 0
    for r, text in zip(regions, translated_texts):
        rect = r["rect"]
        if not text.strip():
            continue
        initial_size = rect.height * 0.75
        fitted_size = fit_fontsize(text, initial_size, rect.width, "helv", min_size=4.0)
        fitted_size = max(4.0, min(fitted_size, rect.height * 0.9))
        origin = pymupdf.Point(rect.x0, rect.y0 + rect.height * 0.78)
        try:
            out_page.insert_text(origin, text, fontsize=fitted_size,
                                  fontname="helv", color=(0, 0, 0))
            count += 1
        except Exception:
            pass

    return count


# ── Main Pipeline Loop ────────────────────────────────────

src_doc = pymupdf.open(INPUT_PDF)
out_doc = pymupdf.open()
out_doc.insert_pdf(src_doc)
src_doc.close()

ocr_engine = None
digital_pages = 0
scanned_pages = 0
skipped_pages = 0

for pg in range(len(out_doc)):
    out_page = out_doc[pg]
    info = analyze_page(out_page)

    if not info["is_scanned"]:
        print(f"\n📄 Page {pg+1}/{len(out_doc)}: {info['page_type']} "
              f"({info['span_count']} spans, {info['drawings']} drawings, "
              f"{info['img_coverage']}% img coverage)")
        count = reconstruct_digital_page(out_page, font_map)
        print(f"   ✅ Redacted & re-inserted {count} spans")
        digital_pages += 1
    else:
        print(f"\n📷 Page {pg+1}/{len(out_doc)}: {info['page_type']} "
              f"({info['span_count']} spans, {info['images']} images)")

        # Skip image-only pages (photos, diagrams) — no text to translate
        if is_content_image(out_page, info):
            print(f"   🖼️  Image-only page detected, preserving as-is")
            skipped_pages += 1
            continue

        if ocr_engine is None:
            if PaddleOCR is None:
                print("   ❌ PaddleOCR not available — skipping. RESTART RUNTIME.")
                scanned_pages += 1
                continue
            print("  ⏳ Initializing PaddleOCR (first scanned page)...")
            ocr_engine = PaddleOCR(use_textline_orientation=True, lang='en')

        count = reconstruct_scanned_page(out_page, ocr_engine, pg)
        print(f"   ✅ OCR'd & inserted {count} text regions")
        scanned_pages += 1

UNIFIED_PDF = str(WORK_DIR / "unified_reconstructed.pdf")
out_doc.save(UNIFIED_PDF)
out_doc.close()

print(f"\n{'='*60}")
print(f"🎉 UNIFIED RECONSTRUCTION COMPLETE")
print(f"   Digital pages:      {digital_pages}")
print(f"   Scanned pages:      {scanned_pages}")
print(f"   Image-only skipped: {skipped_pages}")
print(f"   Output: {UNIFIED_PDF}")
print(f"{'='*60}")

✅ PaddleOCR: available

📷 Page 1/5: OCR'd SCAN (54 spans, 1 images)
  ⏳ Initializing PaddleOCR (first scanned page)...
download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:19<00:00, 203kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:23<00:00, 444kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:01<00:00, 1.24MiB/s]

[2026/02/23 22:10:26] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

[2026/02/23 22:10:27] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2026/02/23 22:10:27] ppocr DEBUG: dt_boxes num : 33, elapsed : 0.2270035743713379
[2026/02/23 22:10:31] ppocr DEBUG: rec_res num  : 33, elapsed : 3.6450681686401367
   ⚠️  Hallucination suspected (ratio=2.8), keeping original: 'INTRODUCTION'
   ✅ No PII found in 33 spans
   ✅ OCR'd & inserted 33 text regions

📄 Page 2/5: DIGITAL (51 spans, 5 drawings, 0.0% img coverage)
   🔍 Running Groq verification on 51 spans...
   🔍 16/47 spans flagged as high-risk
   ✅ Batch 1: corrected 8 spans
   ✅ Batch 2: corrected 8 spans
   🔒 Running PII redaction on spans...
   ✅ No PII found in 51 spans
   ✅ Redacted & re-inserted 51 spans

📄 Page 3/5: DIGITAL (65 spans, 2 drawings, 0.0% img coverage)
   🔍 Running Groq verification on 65 spans...
   🔍 5/60 spans flagged as high-risk
   ✅ Batch 1: corrected 5 spans
   🔒 Running PII redaction on spans...
   ✅ No PII found in 65 